In [1]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_ind

In [2]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

Объединим два массива в DataFrame, предварительно преобразовав в Series

In [3]:
df = pd.concat([pd.Series(petersburg), pd.Series(magadan)], axis=1)
df.rename(columns={0: 'petersburg', 1: 'magadan'}, inplace=True)
df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


заполним NaN медианой по стобцу:

In [4]:
df['petersburg'][7] = round(df['petersburg'].median(), 4)
df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


Посчитаем разницу в среднем размере мидии в зависимости от города-производителя

In [5]:
round(df['petersburg'].mean()-df['magadan'].mean(), 4)

0.0252

уровень степени значимости возьмем в 0.05 (стандартный уровень нам подходит)

проверяем на нормальность распределения

In [6]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

In [7]:
alpha = 0.05
_, p = shapiro(df)

round(p,4)

0.1014

In [8]:
if p > alpha: display(H0)
else: display(Ha)

'Данные распределены нормально'

In [9]:
display(df.corr())

,petersburg,magadan
petersburg,1.000000,0.256665
magadan,0.256665,1.000000


Выбор теста на корреляция выпал на метод Пирсона - тк распределение нормальное, без выбросовочень, корреляция слабая или даже отсутствует

Для проверки значимости разницы в рамерах - используем Независимый T-тест, причина выбора: распределение нормальное, зависимая переменнная количественная, сравниваются ровно 2 группы и элементов в выборке меньше 30 

In [10]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в двух разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в двух разных местах.'

In [11]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['petersburg'], df['magadan'], equal_var=True)
    p = round(test_results[1],3)

    if p>alpha:
        print(f"{p} > {alpha}. \n Мы не можем отвергнуть нулевую гипотезу. \n {H0}")
    else:
        print(f"{p} <= {alpha}. \n Мы отвергаем нулевую гипотезу. {Ha}")

t_test(df)


*** Результаты независимого T-теста ***
0.003 <= 0.05. 
 Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в двух разных местах.
